# Identifying Positives in the Multiplex

This code was used on the images taken by the Leica SP8 to identify regions where all three relevant markers were present, indicating the possible presence of cancer.

In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import cv2

Image.MAX_IMAGE_PIXELS = 933120000


def highlight_roi_boxes(s100_path, sox10_path, ki67_path, save_path, pad):
    """
    Draws a box around regions in the image where all three markers are present.

    Parameters
    ----------
    s100_path : str
        Path to s100 binary image.
    sox10_path : str
        Path to sox10 binary image.
    ki67_path : str
        Path to ki67 binary image.
    save_path : str
        Path to save location of output image.
    pad : int
        Size of padding to draw the box

    """
    s100_img = Image.open(s100_path)
    sox10_img = Image.open(sox10_path)
    ki67_img = Image.open(ki67_path)

    s100_array = np.array(s100_img) / 255
    sox10_array = np.array(sox10_img) / 255
    ki67_array = np.array(ki67_img) / 255

    combined = s100_array + sox10_array + ki67_array
    mask = combined > 2

    coords = np.argwhere(mask)

    highlighted_regions = sox10_img.convert("RGB")
    draw = ImageDraw.Draw(highlighted_regions)
    H, W = mask.shape
    for row, col in coords:
        x = int(col)
        y = int(row)
        left = max(x - pad, 0)
        top = max(y - pad, 0)
        right = min(x + pad, W - 1)
        bottom = min(y + pad, H - 1)
        draw.point([x, y], fill=(0, 255, 0))
        for i in range(5):
            draw.rectangle(
                [left - i, top - i, right - i, bottom - i], outline=(255, 0, 0)
            )

    highlighted_regions.show()
    highlighted_regions.save(save_path)


def highlight_roi_connected_components(
    s100_path,
    sox10_path,
    ki67_path,
    overlay_path,
    save_path,
    pad=50,
    save=False,
    scaled=False,
    min_area=50,
):
    """
    Highlights regions in the image using points or ellipses where all three markers are present.

    Parameters
    ----------
    s100_path : str
        Path to s100 binary image.
    sox10_path : str
        Path to sox10 binary image.
    ki67_path : str
        Path to ki67 binary image.
    overlay_path : str
        Path to image where the regions should be overlayed.
    save_path : str
        Path to save location of output image.
    pad : int
        Size of radius to draw circle point, default 50
    save : bool
        Flag to indicate whether the output should be saved to save_path
    scaled : bool
        Flag to indicate whether the identified regions should be scaled by their area or given a constant size
    min_area : int
        Minimum area of regions where markers are overlapping in order for that region to be visualized, default 50

    """
    s100_img = Image.open(s100_path)
    sox10_img = Image.open(sox10_path)
    ki67_img = Image.open(ki67_path)
    overlay_img = Image.open(overlay_path)

    s100_array = np.array(s100_img) / 255
    sox10_array = np.array(sox10_img) / 255
    ki67_array = np.array(ki67_img) / 255

    combined = s100_array + sox10_array + ki67_array
    mask = (combined > 2).astype(np.uint8) * 255

    num_labels, _, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)

    highlighted_regions = overlay_img.convert("RGB")
    draw = ImageDraw.Draw(highlighted_regions)
    H, W = mask.shape
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area < min_area:
            continue
        if scaled:
            left = max(x, 0)
            top = max(y, 0)
            right = min(x + w, W - 1)
            bottom = min(y + h, H - 1)
        else:
            left = max(x - pad, 0)
            top = max(y - pad, 0)
            right = min(x + pad, W - 1)
            bottom = min(y + pad, H - 1)

        draw.ellipse(
            [(left, top), (right, bottom)], fill="red", outline="black", width=2
        )

    highlighted_regions.show()
    if save:
        highlighted_regions.save(save_path)

In [6]:
s100_path = "../data_imaging/d16_s100.tif"
sox10_path = "../data_imaging/d16_sox10.tif"
ki67_path = "../data_imaging/d16_ki67.tif"

save_path = "../data_imaging/roi/roi_d16_scaled_2.png"
overlay = "../data_imaging/d16.tif"
pad = 50

highlight_roi_connected_components(
    s100_path=s100_path,
    sox10_path=sox10_path,
    ki67_path=ki67_path,
    overlay_path=overlay,
    save_path=save_path,
    save=True,
    scaled=True,
)